# ಕ್ಲೌಡ್‌ನಲ್ಲಿ ಡೇಟಾ ಸೈನ್ಸ್: "ಅಜೂರ್ ಎಂಎಲ್ ಎಸ್‌ಡಿಕೆ" ವಿಧಾನ

## ಪರಿಚಯ

ಈ ನೋಟ್ಬುಕ್‌ನಲ್ಲಿ, ನಾವು ಅಜೂರ್ ಎಂಎಲ್ ಮೂಲಕ ಮಾದರಿಯನ್ನು ತರಬೇತಿ, ನಿಯೋಜನೆ ಮತ್ತು ಬಳಕೆ ಮಾಡಲು ಅಜೂರ್ ಎಂಎಲ್ ಎಸ್‌ಡಿಕೆ ಅನ್ನು ಹೇಗೆ ಬಳಸುವುದು ಎಂದು ಕಲಿಯುತ್ತೇವೆ.

ಪೂರ್ವಾಪೇಕ್ಷಿತಗಳು:
1. ನೀವು ಅಜೂರ್ ಎಂಎಲ್ ವರ್ಕ್‌ಸ್ಪೇಸ್ ಅನ್ನು ರಚಿಸಿದ್ದೀರಿ.
2. ನೀವು [ಹಾರ್ಟ್ ಫೇಲ್ಯೂರ್ ಡೇಟಾಸೆಟ್](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) ಅನ್ನು ಅಜೂರ್ ಎಂಎಲ್‌ಗೆ ಲೋಡ್ ಮಾಡಿದ್ದೀರಿ.
3. ನೀವು ಈ ನೋಟ್ಬುಕ್ ಅನ್ನು ಅಜೂರ್ ಎಂಎಲ್ ಸ್ಟುಡಿಯೋಗೆ ಅಪ್‌ಲೋಡ್ ಮಾಡಿದ್ದೀರಿ.

ಮುಂದಿನ ಹಂತಗಳು:

1. ಇರುವ ವರ್ಕ್‌ಸ್ಪೇಸ್‌ನಲ್ಲಿ ಒಂದು ಪ್ರಯೋಗವನ್ನು ರಚಿಸಿ.
2. ಒಂದು ಕಂಪ್ಯೂಟ್ ಕ್ಲಸ್ಟರ್ ರಚಿಸಿ.
3. ಡೇಟಾಸೆಟ್ ಅನ್ನು ಲೋಡ್ ಮಾಡಿ.
4. AutoMLConfig ಬಳಸಿ AutoML ಅನ್ನು ಸಂರಚಿಸಿ.
5. AutoML ಪ್ರಯೋಗವನ್ನು ಚಾಲನೆ ಮಾಡಿ.
6. ಫಲಿತಾಂಶಗಳನ್ನು ಅನ್ವೇಷಿಸಿ ಮತ್ತು ಉತ್ತಮ ಮಾದರಿಯನ್ನು ಪಡೆಯಿರಿ.
7. ಉತ್ತಮ ಮಾದರಿಯನ್ನು ನೋಂದಣಿ ಮಾಡಿ.
8. ಉತ್ತಮ ಮಾದರಿಯನ್ನು ನಿಯೋಜಿಸಿ.
9. ಎಂಡ್‌ಪಾಯಿಂಟ್ ಅನ್ನು ಬಳಸಿ.

## ಅಜೂರ್ ಮೆಷಿನ್ ಲರ್ನಿಂಗ್ SDK-ನಿರ್ದಿಷ್ಟ ಆಮದುಗಳು


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## ಕಾರ್ಯಸ್ಥಳವನ್ನು ಪ್ರಾರಂಭಿಸಿ
ಸ್ಥಿರಗೊಂಡ ಸಂರಚನೆಯಿಂದ ಕಾರ್ಯಸ್ಥಳ ವಸ್ತುವನ್ನು ಪ್ರಾರಂಭಿಸಿ. .\config.json ನಲ್ಲಿ ಸಂರಚನಾ ಕಡತವು ಇರುವುದನ್ನು ಖಚಿತಪಡಿಸಿಕೊಳ್ಳಿ.


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Azure ML ಪ್ರಯೋಗವನ್ನು ರಚಿಸಿ

ನಾವು ಇತ್ತೀಚೆಗೆ ಪ್ರಾರಂಭಿಸಿದ ವರ್ಕ್‌ಸ್ಪೇಸ್‌ನಲ್ಲಿ 'aml-experiment' ಎಂಬ ಪ್ರಯೋಗವನ್ನು ರಚಿಸೋಣ.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## ಕಂಪ್ಯೂಟ್ ಕ್ಲಸ್ಟರ್ ರಚಿಸಿ
ನೀವು ನಿಮ್ಮ AutoML ರನ್‌ಗಾಗಿ [ಕಂಪ್ಯೂಟ್ ಗುರಿ](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) ರಚಿಸಬೇಕಾಗುತ್ತದೆ.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## ಡೇಟಾ
ನೀವು ಡೇಟಾಸೆಟ್ ಅನ್ನು Azure ML ಗೆ ಅಪ್‌ಲೋಡ್ ಮಾಡಿರುವುದನ್ನು ಖಚಿತಪಡಿಸಿಕೊಳ್ಳಿ ಮತ್ತು ಕೀ ಡೇಟಾಸೆಟ್‌ನ ಅದೇ ಹೆಸರಾಗಿರಬೇಕು.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML ಸಂರಚನೆ


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## AutoML ರನ್


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

## ಅತ್ಯುತ್ತಮ ಮಾದರಿಯನ್ನು ಉಳಿಸಿ


In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## ಅತ್ಯುತ್ತಮ ಮಾದರಿಯನ್ನು ನಿಯೋಜಿಸಿ

ಅತ್ಯುತ್ತಮ ಮಾದರಿಯನ್ನು ನಿಯೋಜಿಸಲು ಕೆಳಗಿನ ಕೋಡ್ ಅನ್ನು ಚಾಲನೆ ಮಾಡಿ. ನಿಯೋಜನೆಯ ಸ್ಥಿತಿಯನ್ನು ನೀವು Azure ML ಪೋರ್ಟಲ್‌ನಲ್ಲಿ ನೋಡಬಹುದು. ಈ ಹಂತಕ್ಕೆ ಕೆಲವು ನಿಮಿಷಗಳು ಬೇಕಾಗಬಹುದು.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## ಎಂಡ್ಪಾಯಿಂಟ್ ಅನ್ನು ಬಳಸಿ
ನೀವು ಕೆಳಗಿನ ಇನ್‌ಪುಟ್ ಮಾದರಿಗೆ ಇನ್‌ಪುಟ್‌ಗಳನ್ನು ಸೇರಿಸಬಹುದು.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕಾರ**:  
ಈ ದಸ್ತಾವೇಜು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
